<a href="https://colab.research.google.com/github/clopesro/Compartilha-livro/blob/main/C%C3%B3pia_de_Imers%C3%A3o_IA_Alura_%2B_Google_Gemini_Aula_05_Agentes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip -q install google-genai

In [2]:
# Configura a API Key do Google Gemini

import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [3]:
# Configura o cliente da SDK do Gemini

from google import genai

client = genai.Client()

MODEL_ID = "gemini-2.0-flash"

In [4]:
# Pergunta ao Gemini uma informação mais recente que seu conhecimento

from IPython.display import HTML, Markdown

# Perguntar pro modelo quando é a próxima imersão de IA ###############################################
resposta = client.models.generate_content(
    model=MODEL_ID,
    contents='Quando é a próxima Imersão IA com Google Gemini da Alura?',
)

# Exibe a resposta na tela
display(Markdown(f"Resposta:\n {resposta.text}"))

Resposta:
 A Alura não tem uma data específica para a próxima Imersão IA com Google Gemini. A melhor forma de ficar por dentro de novos cursos e imersões é:

*   **Seguir a Alura nas redes sociais:** Eles costumam anunciar novidades no LinkedIn, Instagram e outras plataformas.
*   **Assinar a newsletter da Alura:** Assim você recebe as novidades diretamente no seu e-mail.
*   **Ficar de olho na página de "Próximos Eventos" ou "Lançamentos" no site da Alura:** Essas seções são atualizadas com os próximos cursos e imersões.

Além disso, você pode entrar em contato diretamente com a Alura através dos canais de atendimento ao cliente para perguntar sobre a previsão de novas turmas da Imersão IA com Google Gemini.

In [7]:
# Pergunta ao Gemini uma informação utilizando a busca do Google como contexto

response = client.models.generate_content(
    model=MODEL_ID,
    contents='Quando é a próxima Imersão IA com Google Gemini da Alura?',
    config={"tools": [{"google_search": {}}]}
)
display(Markdown(f"Resposta:\n {response.text}"))

Resposta:
 A próxima Imersão IA com Google Gemini da Alura acontecerá entre os dias 12 e 16 de maio de 2025. As inscrições estão abertas até o dia 11 de maio e são totalmente gratuitas.


In [8]:
# Exibe a busca
print(f"Busca realizada: {response.candidates[0].grounding_metadata.web_search_queries}")
# Exibe as URLs nas quais ele se baseou
print(f"Páginas utilizadas na resposta: {', '.join([site.web.title for site in response.candidates[0].grounding_metadata.grounding_chunks])}")
print()
display(HTML(response.candidates[0].grounding_metadata.search_entry_point.rendered_content))

Busca realizada: ['Alura Imersão IA com Google Gemini']
Páginas utilizadas na resposta: alura.com.br, tecmundo.com.br



In [9]:
# Instalar Framework ADK de agentes do Google ################################################
!pip install -q google-adk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 24.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.1/232.1 kB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.1/217.1 kB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 334.1/334.1 kB 25.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.1/125.1 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.0/119.0 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.9/194.9 kB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7

In [10]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")

In [11]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [12]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [20]:
##########################################
# --- Agente 1: Buscador de Notícias --- #
##########################################
# --- Agente 1: Buscador de Notícias --- #
##########################################
def agente_buscador(topico,data_de_hoje):
    buscador = Agent(
        name="agente_buscador",
        model="gemini-2.0-flash",
        description="Agente que busca notícias sobre o tópico indicado",
        tools=[google_search],
        instruction="""
        voce é um assistente de pesquisa. a sua tarefa é usar a ferramenta de busca do google (google_search)
        para recuperar as ultimas notícias de lançamentos muito relevante sobre o tópico abaixo.
        foque em no máximo 5 lançamentos relevantes, com base na quantidade e entusiasmo das notícias sobre eles.
        Se um tema tiver poucas notícias ou reaçoes entusiasmadas, é possível que ele não seja tão relevante assim
        e pode ser substituido por outro que tenha mais.
        """
    )

    entrada_do_agente_buscador = f"Tópico: {topico}\nData: {data_de_hoje}"
    lancamentos = call_agent(buscador, entrada_do_agente_buscador)
    return lancamentos


In [22]:
################################################
# --- Agente 2: Planejador de posts --- #
################################################
def agente_planejador(topico, lancamentos_buscados):
    planejador = Agent(
        name="agente_planejador",
        model="gemini-2.0-flash",
        description="Agente que planeja posts",
        tools=[google_search],
        instruction="""
        voce é um assistente de conteúdo.especialista em redes sociais. Com base na lista de
        lançamentos mais recentes e relevantes buscador, voce deve:
        usar a ferramenta de busca do Google(google_search) para criar um plano sobre
        quais são os pontos mais relevantes que poderiam abordar em um post sobre informaçoes sobre os temas e aprofundar.
        Ao final, voce irá escolher o tema mais relevante entre eles com base nas suas pesquisas
        e retornar esse tema, seus pontos mais relevantes, e um plano com os assuntos a serem abordados no post
        """
    )

    entrada_do_agente_planejador = f"Tópico:{topico}\nLançamentos buscados: {lancamentos_buscados}"
    # Executa o agente
    plano_do_post = call_agent(planejador, entrada_do_agente_planejador)
    return plano_do_post

In [23]:
######################################
# --- Agente 3: Redator do Post --- #
######################################
def agente_redator(topico, plano_de_post):
    redator = Agent(
        name="agente_redator",
        model="gemini-2.5-pro-preview-03-25",
        instruction="""
            Você é um Redator Criativo especializado em criar posts virais para redes sociais.
            Você escreve posts para a empresa Alura, a maior escola online de tecnologia do Brasil.
            Utilize o tema fornecido no plano de post e os pontos mais relevantes fornecidos e, com base nisso,
            escreva um rascunho de post para Instagram sobre o tema indicado.
            O post deve ser engajador, informativo, com linguagem simples e incluir 2 a 4 hashtags no final.
            """,
        description="Agente redator de posts engajadores para Instagram"
    )
    entrada_do_agente_redator = f"Tópico: {topico}\nPlano de post: {plano_de_post}"
    # Executa o agente
    rascunho = call_agent(redator, entrada_do_agente_redator)
    return rascunho

In [24]:
##########################################
# --- Agente 4: Revisor de Qualidade --- #
##########################################
def agente_revisor(topico, rascunho_gerado):
    revisor = Agent(
        name="agente_revisor",
        model="gemini-2.5-pro-preview-03-25",
        instruction="""
            Você é um Editor e Revisor de Conteúdo meticuloso, especializado em posts para redes sociais, com foco no Instagram.
            Por ter um público jovem, entre 18 e 30 anos, use um tom de escrita adequado.
            Revise o rascunho de post de Instagram abaixo sobre o tópico indicado, verificando clareza, concisão, correção e tom.
            Se o rascunho estiver bom, responda apenas 'O rascunho está ótimo e pronto para publicar!'.
            Caso haja problemas, aponte-os e sugira melhorias.
            """,
        description="Agente revisor de post para redes sociais."
    )
    entrada_do_agente_revisor = f"Tópico: {topico}\nRascunho: {rascunho_gerado}"
    # Executa o agente
    texto_revisado = call_agent(revisor, entrada_do_agente_revisor)
    return texto_revisado

In [40]:
data_de_hoje = date.today().strftime("%d/%m/%Y")

print("🚀 Iniciando o Sistema de Criação de Posts para Instagram com 4 Agentes 🚀")

# --- Obter o Tópico do Usuário ---
topico = input("❓ Por favor, digite o TÓPICO sobre o qual você quer criar o post de tendências: ")

# Inserir lógica do sistema de agentes ################################################
if not topico:
  print("voce esqueceu de digitar")
else:
  print(f"Maravilha! Vamos então criar o post sobre novidades em {topico}")

  lancamentos_buscados = agente_buscador(topico, data_de_hoje)
  print("\n---Resultado do Agente 1 (Buscador)---\n")
  display(to_markdown(lancamentos_buscados))
  print("__________________________________________________________________")

  plano_de_post = agente_planejador(topico, lancamentos_buscados)
  print("\n---Resultado do Agente 2 (Planejador)---\n")
  display(to_markdown(plano_de_post))
  print("__________________________________________________________________")


  rascunho_de_post = agente_redator(topico, plano_de_post)
  print("\n---Resultado do Agente 3 (Redator)---\n")
  display(to_markdown(rascunho_de_post))
  print("__________________________________________________________________")


  post_final = agente_revisor(topico,rascunho_de_post)
  print("\n---Resultado do Agente 4 (Revisor)---\n")
  display(to_markdown(post_final))
  print("__________________________________________________________________")



🚀 Iniciando o Sistema de Criação de Posts para Instagram com 4 Agentes 🚀
❓ Por favor, digite o TÓPICO sobre o qual você quer criar o post de tendências: AGENTES DE IA
Maravilha! Vamos então criar o post sobre novidades em AGENTES DE IA

---Resultado do Agente 1 (Buscador)---



> Para fornecer as notícias mais relevantes sobre lançamentos de Agentes de IA, vou realizar algumas buscas no Google. Minhas buscas serão focadas em identificar agentes de IA que receberam cobertura noticiosa significativa e demonstrações de entusiasmo, indicando sua relevância.
> 
> 
> Com base nas minhas buscas, aqui estão alguns dos lançamentos e tendências mais relevantes no espaço de Agentes de IA até maio de 2025:
> 
> 1.  **Agentes de IA Autônomos:** Empresas como Amazon, Google e Apple estão na vanguarda do desenvolvimento de agentes de IA capazes de realizar tarefas complexas de forma independente, acessando sistemas internos e dados proprietários para automação avançada (Exame). A Anthropic também destaca que os agentes de IA como Claude estão se tornando mais eficazes, indo além do preenchimento automático para depurar e corrigir códigos (MIT Technology Review).
> 
> 2.  **Aplicações no Mercado de Trabalho:** Há uma expectativa crescente de que os agentes de IA comecem a "entrar no mercado de trabalho" em 2025, impactando materialmente os resultados das empresas (MIT Technology Review). Esses agentes estão sendo aprimorados para usar ferramentas com mais eficácia, lidar com tarefas complexas e reconhecer erros que exigem feedback do usuário.
> 
> 3.  **Agentes de IA para Criptomoedas:** Agentes de IA estão sendo utilizados no mundo das criptomoedas para aprimorar a negociação, automatizar tarefas e ajudar a gerenciar comunidades descentralizadas (Airdrop Alert). Alguns exemplos incluem AIXBT da Virtuals, especializado em negociação de criptomoedas, e AI16z, focado na previsão do desempenho de tokens.
> 
> 4.  **Plataformas e Ferramentas para Agentes de IA:** Estão surgindo novas plataformas e ferramentas para facilitar o desenvolvimento e a implantação de agentes de IA. Modelos de linguagem como GPT-4 da OpenAI e Claude da Anthropic são a base do raciocínio desses agentes (RDD10+).
> 
> 5.  **Adoção Empresarial:** Relatórios da Deloitte e Gartner sugerem que um número significativo de empresas planeja integrar agentes autônomos até 2025, com estimativas de que 25% das empresas adotarão essas ferramentas este ano e 50% até 2027 (Blog da Engenharia).
> 
> Além disso, vale mencionar que a OpenAI lançou o Codex, um novo agente de IA para usuários do ChatGPT, projetado para agilizar o desenvolvimento de software, com a capacidade de escrever recursos de software, corrigir bugs e executar testes (Bloomberg Linea).
> 


__________________________________________________________________

---Resultado do Agente 2 (Planejador)---



> Com base nas informações fornecidas, vamos criar um plano de conteúdo focado em Agentes de IA. Para isso, usaremos a ferramenta de busca do Google para aprofundar alguns pontos e identificar o tema mais relevante para um post.
> 
> 
> Após analisar as informações e os resultados da busca, o tema mais relevante para um post sobre Agentes de IA é:
> 
> **Tema:** A Transformação do Mercado de Trabalho e das Empresas com Agentes de IA em 2025.
> 
> **Pontos Relevantes:**
> 
> *   **Adoção Acelerada:** Empresas de diversos setores estão integrando agentes de IA para otimizar processos e simplificar tarefas. A expectativa é que essa adoção cresça exponencialmente até 2027 (Blog da Engenharia, IT Forum, The Shift).
> *   **Impacto no Mercado de Trabalho:** Agentes de IA estão "entrando no mercado de trabalho", impactando materialmente os resultados das empresas. Eles estão sendo aprimorados para usar ferramentas com mais eficácia, lidar com tarefas complexas e reconhecer erros (MIT Technology Review). Estima-se que 85 milhões de empregos poderão ser substituídos por automação até 2025, mas 97 milhões de novas funções surgirão (Rede Líderes Digitais).
> *   **Novas Funções e Gestão:** A integração de agentes de IA cria a necessidade de novas funções, como "gestor de agentes", para construir, delegar e gerenciar esses agentes (PwC, Microsoft News).
> *   **Aplicações Diversificadas:** Agentes de IA estão sendo utilizados em diversas áreas, como e-commerce, atendimento ao cliente, marketing, finanças, criptomoedas e desenvolvimento de software (Exame, IT Forum, Airdrop Alert).
> *   **Plataformas e Ferramentas:** Surgimento de plataformas e ferramentas para facilitar o desenvolvimento e a implantação de agentes de IA, como AutoGen, CrewAI, LangChain e Vertex AI Agent Builder (Skim AI).
> *   **Autonomia e Eficiência:** Agentes de IA estão se tornando mais autônomos, capazes de agir por conta própria, aprender com o ambiente e tomar decisões em tempo real (RDD10+, The Shift).
> *   **Investimentos:** Bilhões de dólares estão sendo investidos em ferramentas que facilitam a construção de agentes autônomos (The Shift, Meio e Mensagem).
> *   **Segurança:** A segurança dos agentes é essencial, especialmente contra ataques como "prompt injection" (MIT Technology Review).
> 
> **Plano de Conteúdo:**
> 
> 1.  **Título:** Agentes de IA: A Revolução no Trabalho e nas Empresas em 2025
> 
> 2.  **Introdução:**
> 
>     *   O que são Agentes de IA e por que eles estão ganhando destaque em 2025.
>     *   A previsão de Sam Altman sobre agentes de IA "entrando no mercado de trabalho".
>     *   Adoção acelerada por empresas de diversos setores.
> 
> 3.  **O Impacto no Mercado de Trabalho:**
> 
>     *   Como os agentes de IA estão transformando as funções tradicionais.
>     *   A criação de novas funções, como "gestor de agentes".
>     *   Oportunidades e desafios da colaboração entre humanos e agentes de IA.
>     *   Estudo do World Economic Forum sobre a substituição e criação de empregos.
> 
> 4.  **Aplicações Práticas em Diversos Setores:**
> 
>     *   E-commerce e atendimento ao cliente: chatbots avançados e personalização.
>     *   Marketing e finanças: análise de dados e automação de campanhas.
>     *   Criptomoedas: negociação automatizada e gestão de comunidades.
>     *   Desenvolvimento de software: assistentes de programação e depuração de código.
> 
> 5.  **Plataformas e Ferramentas para Desenvolver Agentes de IA:**
> 
>     *   Destaque para plataformas como AutoGen, CrewAI, LangChain e Vertex AI Agent Builder.
>     *   A importância de modelos de linguagem como GPT-4 e Claude.
>     *   Recursos e funcionalidades essenciais para o desenvolvimento de agentes de IA.
> 
> 6.  **O Futuro dos Agentes de IA:**
> 
>     *   A crescente autonomia e capacidade de aprendizado dos agentes.
>     *   A importância da segurança e da ética no desenvolvimento de agentes de IA.
>     *   O potencial de transformação em diversos setores e na sociedade como um todo.
>     *   A necessidade de requalificação profissional para acompanhar a evolução da IA.
> 
> 7.  **Conclusão:**
> 
>     *   Os agentes de IA como catalisadores da transformação digital.
>     *   A importância de se preparar para um futuro com agentes de IA no mercado de trabalho.
>     *   Oportunidades para empresas e profissionais que souberem aproveitar essa tecnologia.
> 
> Este plano de conteúdo aborda os pontos mais relevantes sobre Agentes de IA, fornecendo informações valiosas e insights sobre o impacto dessa tecnologia no mercado de trabalho e nas empresas em 2025.
> 


__________________________________________________________________


...


---Resultado do Agente 3 (Redator)---



__________________________________________________________________


Exception in thread Thread-77 (_asyncio_thread_main):
Traceback (most recent call last):
  File "/usr/lib/python3.11/threading.py", line 1045, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.11/threading.py", line 982, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.11/dist-packages/google/adk/runners.py", line 138, in _asyncio_thread_main
    asyncio.run(_invoke_run_async())
  File "/usr/lib/python3.11/asyncio/runners.py", line 190, in run
    return runner.run(main)
           ^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/asyncio/runners.py", line 118, in run
    return self._loop.run_until_complete(task)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/asyncio/base_events.py", line 654, in run_until_complete
    return future.result()
           ^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/google/adk/runners.py", line 126, in _invoke_run_async
    async for event in self.run_async(
  File "/u


---Resultado do Agente 4 (Revisor)---



__________________________________________________________________


SyntaxError: invalid syntax (<ipython-input-27-b1be3ec28eff>, line 1)